In [1]:
!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
data['week_no'].nunique()

95

In [5]:
data.shape

(2396804, 12)

In [6]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [7]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [8]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [9]:
item_features = pd.read_csv('product.csv')
item_features.head(2)

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [10]:
user_features = pd.read_csv('hh_demographic.csv')
user_features.head(2)

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### Train-test split

In [11]:
data['week_no'].max()

95

In [12]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [13]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

In [14]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [15]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [16]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [17]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [18]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

Wall time: 2.51 s


In [19]:
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]"


### 1.2 Popularity-based recommendation

In [20]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

In [21]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

Wall time: 67.1 ms


In [22]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender (Домашнее задание - 1 задача)

In [23]:
import random

def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights)  # p>0
    
    return recs

In [24]:
%%time

items_weights= data_train[['item_id','sales_value']]

# При x-> 0, log(x)-> минус бесконечность, поэтому нуль необходимо заменить на не нулевое значение
items_weights.loc[items_weights['sales_value'] == 0, 'sales_value'] = 0.001

items_weights['weight'] = np.log(items_weights['sales_value'])

# Значения weight < 0, заменяем на 0
items_weights.loc[(items_weights['weight'] < 0)] = 0

# items_weights
min_value = items_weights['weight'].min()
max_value = items_weights['weight'].max()
summ = items_weights['weight'].sum()
items_weights = items_weights[['item_id','weight']].groupby('item_id').sum()

# Сумма weight по всем товарам = 1
items_weights['weight'] = items_weights['weight']/summ

items_weights['item_id'] = items_weights.index  # Преобразуем индек в поле 'item_id'
# items_weights.head(3)

Wall time: 177 ms


D:\Anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<timed exec>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9881690, 1015907, 5567954, 1113438, 1118244]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9707040, 1041962, 883003, 12262778, 1034176]"


# 2. Детерминированные алгоритмы item-item

In [26]:
data.item_id.nunique()

89051

In [27]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [28]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [29]:
len(top_5000)

5000

In [30]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [31]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[(~data_train['item_id'].isin(top_5000)), 'item_id'] = 999999

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [32]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
sparse_user_item

<2499x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 667080 stored elements in Compressed Sparse Row format>

In [34]:
user_item_matrix.shape

(2499, 5001)

In [35]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [36]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [37]:
user_item_matrix

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

Wall time: 435 ms


In [39]:
data_test.user_id.describe()

count    118314.000000
mean       1274.589499
std         718.479723
min           1.000000
25%         679.000000
50%        1282.000000
75%        1917.000000
max        2500.000000
Name: user_id, dtype: float64

In [40]:
userid_to_id[13]

12

In [41]:
recs = model.recommend(userid=userid_to_id[13],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

In [42]:
recs

[(2381, 98195.0),
 (3408, 90149.0),
 (2148, 72193.0),
 (3587, 19332.0),
 (3947, 19126.0)]

In [43]:
id_to_itemid[recs[1][0]]

1082185

In [44]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1082185, 981760, 1098066, 1127831]

In [45]:
itemid_to_id[999999]

2381

In [46]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[ itemid_to_id[999999] ],
                                    recalculate_user=True)
                    ])

Wall time: 28 ms


In [47]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9881690, 1015907, 5567954, 1113438, 1118244]","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9707040, 1041962, 883003, 12262778, 1034176]","[1082185, 981760, 1098066, 995242, 826249]"


### (Домашнее задание - 2 задача)

In [48]:
popularity = data_train.groupby('item_id')['quantity', 'sales_value'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

C:\Users\4DF2~1\AppData\Local\Temp/ipykernel_11496/1202125765.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  popularity = data_train.groupby('item_id')['quantity', 'sales_value'].sum().reset_index()


In [49]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [50]:
top_5000[:2]

[6534178, 6533889]

In [51]:
weighted_top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)

In [52]:
weighted_top_5000.head(2)

,item_id,n_sold,sales_value
4346,6534178,190227964,447799.94
4339,6533889,15978434,40483.34


In [53]:
def random_recommendation_5000(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [54]:
%%time

items = weighted_top_5000.item_id.unique()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation_5000(top_5000, n=5))

result.head(2)

Wall time: 507 ms


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9881690, 1015907, 5567954, 1113438, 1118244]","[1082185, 981760, 1127831, 995242, 1098066]","[911311, 1038560, 977798, 1053612, 1059473]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9707040, 1041962, 883003, 12262778, 1034176]","[1082185, 981760, 1098066, 995242, 826249]","[956995, 10285106, 5569379, 862981, 1041476]"


In [55]:
def popularity_recommendation_5000(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [56]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation_5000(data_train, n=5)

result['popular_recommendation_5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 35 ms


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,random_recommendation_5000,popular_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9881690, 1015907, 5567954, 1113438, 1118244]","[1082185, 981760, 1127831, 995242, 1098066]","[911311, 1038560, 977798, 1053612, 1059473]","[999999, 6534178, 6533889, 1029743, 6534166]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9707040, 1041962, 883003, 12262778, 1034176]","[1082185, 981760, 1098066, 995242, 826249]","[956995, 10285106, 5569379, 862981, 1041476]","[999999, 6534178, 6533889, 1029743, 6534166]"


In [57]:
import random

def weighted_random_recommendation_5000(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, p=weights)  # p>0
    
    return recs

In [58]:
%%time

items_weights_5000 = weighted_top_5000[['item_id','sales_value']]

# При x-> 0, log(x)-> -oo, поэтому 0 заменяем на не нулевое значение
items_weights_5000.loc[items_weights_5000['sales_value'] == 0, 'sales_value'] = 0.001

items_weights_5000['weight'] = np.log(items_weights_5000['sales_value'])

# Значения weight < 0, заменяем на 0
items_weights_5000.loc[(items_weights_5000['weight'] < 0)] = 0

# items_weights
min_value = items_weights_5000['weight'].min()
max_value = items_weights_5000['weight'].max()
summ = items_weights_5000['weight'].sum()
items_weights_5000 = items_weights_5000[['item_id','weight']].groupby('item_id').sum()

# Сумма weight по всем товарам = 1
items_weights_5000['weight'] = items_weights_5000['weight']/summ

items_weights_5000['item_id'] = items_weights_5000.index  # Преобразуем индек в поле 'item_id'
# items_weights.head(2)

Wall time: 5 ms


D:\Anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [59]:
result['weighted_random_recommendation_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation_5000(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation,itemitem,random_recommendation_5000,popular_recommendation_5000,weighted_random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1886755, 13007248, 6773053, 304162, 8090521]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9881690, 1015907, 5567954, 1113438, 1118244]","[1082185, 981760, 1127831, 995242, 1098066]","[911311, 1038560, 977798, 1053612, 1059473]","[999999, 6534178, 6533889, 1029743, 6534166]","[933835, 1015678, 871754, 1082833, 1092810]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1163953, 9488055, 1123045, 880530, 988421]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9707040, 1041962, 883003, 12262778, 1034176]","[1082185, 981760, 1098066, 995242, 826249]","[956995, 10285106, 5569379, 862981, 1041476]","[999999, 6534178, 6533889, 1029743, 6534166]","[978269, 1054135, 1119761, 1022003, 1043338]"


In [61]:
# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
#from src.metrics import precision_at_k, recall_at_k


In [62]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
        
    return precision

In [63]:
# Проверим какой алгоритм лучше. 
column_names = [
    'weighted_random_recommendation',
                'weighted_random_recommendation_5000',
                'random_recommendation',
                'random_recommendation_5000',
                'popular_recommendation',
                'popular_recommendation_5000',
          ]
for column_name in column_names:
    precision = result.apply(lambda x: precision_at_k(x[column_name], x['actual'],  5), axis=1).mean() 
    print(f'{precision:.5}: {column_name}')   

0.017336: weighted_random_recommendation
0.017728: weighted_random_recommendation_5000
0.0006856: random_recommendation
0.0067581: random_recommendation_5000
0.15524: popular_recommendation
0.085602: popular_recommendation_5000


### Вывод

Качество бейзлайнов меняется.popular_recommendation ухудшается, остальные улучшаются. Увеличение качества в random_recommendation можно объяснить тем, что при отсечении непопулярных товаров, мы уменьшаем вероятность рандомного ложного выбора. В случае с popular_recommendation - часть попадает не в топ 5000, что снижает точность модели  